In [2]:
library(tidyverse)
library(RPostgreSQL)
library(lubridate)

Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages ---------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats
Loading required package: DBI

Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date



In [3]:
drv <- dbDriver("PostgreSQL")

In [4]:
user <- pw <- "datathon"
dbname <- "mimic"

In [5]:
con <- dbConnect(drv, dbname = dbname, user = user, password = pw)

In [6]:
que <- "
    SELECT * FROM (SELECT subject_id, hadm_id, icustay_id, intime, outtime,
        MIN(intime) over (PARTITION BY subject_id) AS first
        FROM icustays) AS icu
    WHERE intime = first
    LIMIT 10;
"
quedf <- dbGetQuery(con, que)
quedf

subject_id,hadm_id,icustay_id,intime,outtime,first
2,163353,243653,2138-07-17 21:20:07,2138-07-17 23:32:21,2138-07-17 21:20:07
3,145834,211552,2101-10-20 19:10:11,2101-10-26 20:43:09,2101-10-20 19:10:11
4,185777,294638,2191-03-16 00:29:31,2191-03-17 16:46:31,2191-03-16 00:29:31
5,178980,214757,2103-02-02 06:04:24,2103-02-02 08:06:00,2103-02-02 06:04:24
6,107064,228232,2175-05-30 21:30:54,2175-06-03 13:39:54,2175-05-30 21:30:54
7,118037,278444,2121-05-23 15:35:29,2121-05-23 22:01:00,2121-05-23 15:35:29
8,159514,262299,2117-11-20 12:36:10,2117-11-21 14:24:55,2117-11-20 12:36:10
9,150750,220597,2149-11-09 13:07:02,2149-11-14 20:52:14,2149-11-09 13:07:02
10,184167,288409,2103-06-28 11:39:05,2103-07-06 13:51:43,2103-06-28 11:39:05
11,194540,229441,2178-04-16 06:19:32,2178-04-17 20:21:05,2178-04-16 06:19:32


In [7]:
first_icu_que <- "
    SELECT icu.subject_id, icu.hadm_id, icu.icustay_id, icu.intime, icu.outtime,
        adm.admittime, adm.dischtime, adm.deathtime
    FROM icustays AS icu LEFT OUTER JOIN admissions AS adm
    ON adm.subject_id = icu.subject_id AND adm.hadm_id = icu.hadm_id;
"

In [8]:
first_icu <- dbGetQuery(con, first_icu_que)
head(first_icu)

subject_id,hadm_id,icustay_id,intime,outtime,admittime,dischtime,deathtime
268,110404,280836,2198-02-14 23:27:38,2198-02-18 05:26:11,2198-02-11 13:40:00,2198-02-18 03:55:00,2198-02-18 03:55:00
269,106296,206613,2170-11-05 11:05:29,2170-11-08 17:46:57,2170-11-05 11:04:00,2170-11-27 18:00:00,NA
270,188028,220345,2128-06-24 15:05:20,2128-06-27 12:32:29,2128-06-23 18:26:00,2128-06-27 12:31:00,NA
272,164716,210407,2186-12-25 21:08:04,2186-12-27 12:01:13,2186-12-25 21:06:00,2187-01-02 14:57:00,NA
273,158689,241507,2141-04-19 06:12:05,2141-04-20 17:52:11,2141-04-19 06:11:00,2141-04-20 17:00:00,NA
274,130546,254851,2114-06-28 22:28:44,2114-07-07 18:01:16,2114-06-28 22:16:00,2114-07-12 15:30:00,NA


In [9]:
# dbDisconnect(con)
# dbUnloadDriver(drv)

In [10]:
fst_icu <- first_icu %>%
    mutate(intime = ymd_hms(intime),
           outtime = ymd_hms(outtime),
           admittime = ymd_hms(admittime),
           dischtime = ymd_hms(dischtime),
           deathtime = ymd_hms(deathtime))
head(fst_icu)

subject_id,hadm_id,icustay_id,intime,outtime,admittime,dischtime,deathtime
268,110404,280836,2198-02-14 23:27:38,2198-02-18 05:26:11,2198-02-11 13:40:00,2198-02-18 03:55:00,2198-02-18 03:55:00
269,106296,206613,2170-11-05 11:05:29,2170-11-08 17:46:57,2170-11-05 11:04:00,2170-11-27 18:00:00,NA
270,188028,220345,2128-06-24 15:05:20,2128-06-27 12:32:29,2128-06-23 18:26:00,2128-06-27 12:31:00,NA
272,164716,210407,2186-12-25 21:08:04,2186-12-27 12:01:13,2186-12-25 21:06:00,2187-01-02 14:57:00,NA
273,158689,241507,2141-04-19 06:12:05,2141-04-20 17:52:11,2141-04-19 06:11:00,2141-04-20 17:00:00,NA
274,130546,254851,2114-06-28 22:28:44,2114-07-07 18:01:16,2114-06-28 22:16:00,2114-07-12 15:30:00,NA


In [11]:
raw_mort <- fst_icu %>%
    mutate(time_till_death = interval(intime, deathtime) / dhours(1))
head(raw_mort)

subject_id,hadm_id,icustay_id,intime,outtime,admittime,dischtime,deathtime,time_till_death
268,110404,280836,2198-02-14 23:27:38,2198-02-18 05:26:11,2198-02-11 13:40:00,2198-02-18 03:55:00,2198-02-18 03:55:00,76.45611
269,106296,206613,2170-11-05 11:05:29,2170-11-08 17:46:57,2170-11-05 11:04:00,2170-11-27 18:00:00,NA,NA
270,188028,220345,2128-06-24 15:05:20,2128-06-27 12:32:29,2128-06-23 18:26:00,2128-06-27 12:31:00,NA,NA
272,164716,210407,2186-12-25 21:08:04,2186-12-27 12:01:13,2186-12-25 21:06:00,2187-01-02 14:57:00,NA,NA
273,158689,241507,2141-04-19 06:12:05,2141-04-20 17:52:11,2141-04-19 06:11:00,2141-04-20 17:00:00,NA,NA
274,130546,254851,2114-06-28 22:28:44,2114-07-07 18:01:16,2114-06-28 22:16:00,2114-07-12 15:30:00,NA,NA


In [12]:
raw_mort %>%
    pull(time_till_death) %>%
    str

 num [1:61532] 76.5 NA NA NA NA ...


In [14]:
raw_mort_chr <- raw_mort %>%
    select(subject_id, icustay_id, intime, deathtime, time_till_death) %>%
    mutate(intime = as.character(intime), deathtime = as.character(deathtime))
head(raw_mort_chr)

subject_id,icustay_id,intime,deathtime,time_till_death
268,280836,2198-02-14 23:27:38,2198-02-18 03:55:00,76.45611
269,206613,2170-11-05 11:05:29,NA,NA
270,220345,2128-06-24 15:05:20,NA,NA
272,210407,2186-12-25 21:08:04,NA,NA
273,241507,2141-04-19 06:12:05,NA,NA
274,254851,2114-06-28 22:28:44,NA,NA


In [15]:
data.table::fwrite(raw_mort_chr, "/Data/common/raw_mort.csv")

In [ ]:
time_till_death <- raw_mort %>%
    select(subject_id, icustay_id, time_till_death)
head(time_till_death)
nrow(time_till_death)

In [ ]:
data.table::fwrite(time_till_death, "/Data/common/time_till_death.csv")

In [ ]:
mort_icu <- fst_icu %>%
    mutate(onemth = intime + months(1),
           thrmth = intime + months(3),
           sixmth = intime + months(6))
head(mort_icu)

In [ ]:
mort_fst_icu <- mort_icu %>%
    mutate(icu_mort = (!is.na(deathtime) & deathtime >= outtime & deathtime <= outtime)) %>%
    mutate(h_mort = (!is.na(deathtime) & (deathtime >= admittime & deathtime <= dischtime | icu_mort))) %>%
    mutate(one_mort = (!is.na(deathtime) & deathtime <= onemth)) %>%
    mutate(thr_mort = (!is.na(deathtime) & deathtime <= thrmth)) %>%
    mutate(six_mort = (!is.na(deathtime) & deathtime <= sixmth))
head(mort_fst_icu)

In [ ]:
mort_sum <- mort_fst_icu %>%
    select_(.dots = c("subject_id", "hadm_id", "icustay_id", grep("mort", names(.), value = TRUE))) %>%
    mutate_(.dots = grep("mort", names(.), value = TRUE) %>%
                setNames(., .) %>%
                map(~ sprintf("~ as.integer(%s)", .x)) %>%
                map(~ as.formula(.x)))
head(mort_sum)

In [ ]:
# data.table::fwrite(mort_sum, "/Data/common/mort_sum_full.csv")

In [ ]:
mort_sum %>%
    summarise(i = sum(icu_mort, na.rm = TRUE) / n(),
              h = sum(h_mort, na.rm = TRUE) / n(),
              o = sum(one_mort, na.rm = TRUE) / n(),
              t = sum(thr_mort, na.rm = TRUE) / n(),
              s = sum(six_mort, na.rm = TRUE) / n())

In [ ]:
1 + 1